In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
# import MetaTrader5 as mt5
# # from vectorbt.portfolio import Portfolio as vbt_Portfolio

# Connection

In [ ]:
# RICO_REAL   = { 'login':16331371,   'password':'#582640Zz', 'server':'Rico-PRD'                 }
# CLEAR_REAL  = { 'login':1018156111, 'password':'#582640Zz', 'server':'ClearInvestimentos-CLEAR' }

# PATH_7 = r'C:\Program Files\MetaTrader 5 _7\terminal64.exe'

# assert mt5.initialize(PATH_7, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$'] # ['WDO$']
TFRAMES = ['M5','M15','H1','H4','D1']

# Data

In [ ]:
# def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
#     Pipe = []
#     for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
#         timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
#         Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
#         Raw.insert(0, 'a',          Raw.index + 1)
#         Raw.insert(1, 'z',          Raw.index - Raw.index.max())
#         Raw.insert(2, 'symbol',     symbol)
#         Raw.insert(3, 'tf',         TF)
#         Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
#         Pipe.append(Raw)
#     pass
#     return pd.concat(Pipe, ignore_index=1)

# Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
# Raw.head()

In [ ]:
# Raw.to_csv('Storage/Raw.csv', index=False)
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['D1'])
Query.head()

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):

    def _toint(x):
        try:    return int(x)
        except: return x

    Calc = pd.DataFrame()
    Calc['Symbol']  = Src['Symbol']
    Calc['TF']      = Src['TF']

    WINS = [0, 100, 200, 300, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000, inf]
    for (a,b) in [x for x in pd.Series(WINS).rolling(2) if (len(x) == 2)]:
        Calc[f'Chg Abs {_toint(a)}-{_toint(b)}'] = Src.groupby(['Symbol','TF'], sort=0)['Change'].expanding().apply(lambda X: ((a <= X.abs()) & (X.abs() < b)).sum() / X.count() *100)  .reset_index(drop=1)

    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [ ]:
pd.Series(Calc.groupby(['Symbol','TF'], sort=0).tail(1).round(2).to_dict('records')[0])

Symbol                WIN$
TF                      D1
Chg Abs 0-100          4.9
Chg Abs 100-200       5.54
Chg Abs 200-300       4.26
Chg Abs 300-500       9.96
Chg Abs 500-800      12.69
Chg Abs 800-1000     10.68
Chg Abs 1000-1200     7.31
Chg Abs 1200-1500     10.2
Chg Abs 1500-1800     8.35
Chg Abs 1800-2000     4.42
Chg Abs 2000-2200     3.29
Chg Abs 2200-2500     4.74
Chg Abs 2500-2800     3.61
Chg Abs 2800-3000     1.29
Chg Abs 3000-3200     1.85
Chg Abs 3200-3500     2.09
Chg Abs 3500-3800     1.61
Chg Abs 3800-4000     0.48
Chg Abs 4000-4200     0.72
Chg Abs 4200-4500     0.56
Chg Abs 4500-4800      0.4
Chg Abs 4800-5000     0.24
Chg Abs 5000-inf       0.8
dtype: object